In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import pickle
import math

from itertools import product
import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import itertools
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import cv2
import json
from sklearn.model_selection import train_test_split
from functools import partial
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

<ipython-input-2-8549c943defc>:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
class BiLSTMDecoder(nn.Module):
    def __init__(self,seq_len, input_size, hidden_size, linear_filters,embedding_size:int, num_layers = 1,bidirectional=True,device="cpu"):
        super(BiLSTMDecoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = device
        self.num_layers = num_layers
        self.linear_filters = linear_filters[::-1]
        self.embedding_size = embedding_size
        self.bidirectional = bidirectional
        self.seq_len = seq_len

        if bidirectional:
            self.input_linear = nn.Linear(self.embedding_size,4*self.hidden_size)
        else:
            self.input_linear = nn.Linear(self.embedding_size,2*self.hidden_size)

        # define LSTM layer
        self.layers = []
        # add lstm
        self.lstm = nn.LSTM(input_size = self.linear_filters[0], hidden_size = self.hidden_size,
                            num_layers = self.num_layers, bidirectional=True,
                            batch_first=bidirectional)

                        
        # add linear layers 
        if bidirectional:
            self.layers.append(nn.Linear(2*hidden_size,self.linear_filters[0]))
        else:
            self.layers.append(nn.Linear(hidden_size,self.linear_filters[0]))

        for __id,layer_in in enumerate(self.linear_filters):
            if __id == len(linear_filters)-1:
                self.layers.append(nn.Linear(layer_in,self.input_size))
            else:
                self.layers.append(nn.Linear(layer_in,self.linear_filters[__id+1]))

        self.net = nn.Sequential(*self.layers)

        
        

    def forward(self,encoder_hidden):
        """
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence; hidden gives the hidden state and cell state for the last element in the sequence
        """
        
        
        hidden_shape = encoder_hidden.shape
        encoder_hidden = self.input_linear(encoder_hidden)
        
        if self.bidirectional:
            hidden = encoder_hidden.view((-1,4,self.hidden_size))
            hidden = torch.transpose(hidden,1,0)
            h1,h2,c1,c2 = torch.unbind(hidden,0)
            h,c = torch.stack((h1,h2)),torch.stack((c1,c2))
            bs = h.size()[1]
        else:
            hidden = encoder_hidden.view((-1,2,self.hidden_size))
            hidden = torch.transpose(hidden,1,0)
            h,c = torch.unbind(hidden,0)
            bs = h.size()[1]
        
        dummy_input = torch.rand((bs,self.seq_len,self.hidden_size), requires_grad=True).to(self.device)
        
        lstm_out, self.hidden = self.lstm(dummy_input,(h,c))
        x = self.net(lstm_out)
        
        return x

In [ ]:
def get_config(file_loc,device):
    file = torch.load(file_loc,map_location=device)
    return file["model_state_dict"], file["model_config"], file["config"]

model_params, model_config, config = get_config(
    "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/SkeletonAE/model_saves/temp_NTURGB120_skeleton_SGN_classifier_emb1d/40__epoch50_emb1024_xy_decoder.pt",
    torch.device("cpu")
    )

In [ ]:
config

{'n_epochs': 20,
 'model_name': 'BidirectionalLSTM',
 'model': {'num_joint': 12,
  'seq_len': 60,
  'decoder_hidden_size': 1024,
  'linear_filters': [128, 256, 512, 1024],
  'embedding_size': 1024,
  'num_classes': 82,
  'num_layers': 1,
  'is_3d': False,
  'bidirectional': True,
  'batch_size': 32,
  'dev': device(type='cuda'),
  'encoder_hidden_size': 512,
  'input_size': 24},
 'lr': 0.0001,
 'alpha_target': 0.9}

In [7]:
decoder = BiLSTMDecoder(
    seq_len=60,
    input_size=24,
    hidden_size=256,
    linear_filters=[64, 128, 200, 256],
    embedding_size=512,
    num_layers = 2,
    bidirectional=True,
    dev=torch.device("cpu")
    )

In [8]:
# decoder.load_state_dict(model_params)

In [9]:
decoder(torch.rand((32,512))).size()

RuntimeError: ignored

In [ ]:
decoder(torch.rand((32,512)))

tensor([[[ 0.0337,  0.0506, -0.0427,  ...,  0.0586,  0.0806, -0.1121],
         [ 0.0347,  0.0420, -0.0460,  ...,  0.0573,  0.0773, -0.1174],
         [ 0.0338,  0.0467, -0.0415,  ...,  0.0622,  0.0726, -0.1183],
         ...,
         [ 0.0317,  0.0453, -0.0459,  ...,  0.0679,  0.0797, -0.1155],
         [ 0.0320,  0.0495, -0.0406,  ...,  0.0613,  0.0745, -0.1157],
         [ 0.0348,  0.0519, -0.0459,  ...,  0.0548,  0.0758, -0.1201]],

        [[ 0.0407,  0.0497, -0.0440,  ...,  0.0596,  0.0719, -0.1142],
         [ 0.0400,  0.0440, -0.0406,  ...,  0.0634,  0.0719, -0.1156],
         [ 0.0363,  0.0440, -0.0380,  ...,  0.0603,  0.0738, -0.1206],
         ...,
         [ 0.0334,  0.0468, -0.0444,  ...,  0.0633,  0.0805, -0.1250],
         [ 0.0352,  0.0495, -0.0404,  ...,  0.0557,  0.0776, -0.1248],
         [ 0.0378,  0.0461, -0.0480,  ...,  0.0500,  0.0768, -0.1221]],

        [[ 0.0424,  0.0481, -0.0425,  ...,  0.0598,  0.0732, -0.1109],
         [ 0.0411,  0.0464, -0.0447,  ...,  0

In [ ]:
decoder.to(torch.device("cuda"))

BiLSTMDecoder(
  (input_linear): Linear(in_features=512, out_features=1024, bias=True)
  (lstm): LSTM(256, 256, batch_first=True, bidirectional=True)
  (net): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=24, bias=True)
  )
)

In [ ]:
decoder.dev = torch.device("cuda")

In [ ]:
decoder(torch.rand((32,512)).to(torch.device("cuda"))).size()

torch.Size([32, 60, 24])

In [ ]:
decoder(torch.rand((32,512)).to(torch.device("cuda")))

tensor([[[-6.7592e-02, -2.7603e-03,  7.8295e-02,  ..., -4.9465e-02,
           1.5016e-02, -1.1841e-02],
         [-7.5019e-02, -1.1389e-02,  8.9882e-02,  ..., -5.3531e-02,
           2.6554e-02, -2.5699e-02],
         [-7.1981e-02, -7.9594e-03,  9.3056e-02,  ..., -4.1591e-02,
           2.2789e-02, -2.6175e-02],
         ...,
         [-4.0325e-02, -4.6214e-03,  1.0823e-01,  ..., -3.3361e-02,
           7.5475e-03, -1.1074e-02],
         [-4.1810e-02,  3.3957e-03,  1.2069e-01,  ..., -5.2681e-02,
           2.7092e-02, -1.7787e-02],
         [-5.3446e-02,  1.0119e-02,  1.2551e-01,  ..., -5.2466e-02,
           3.2189e-02, -1.1812e-02]],

        [[-8.2152e-02, -1.9085e-02,  7.7862e-02,  ..., -2.7000e-02,
           3.1774e-02, -1.3102e-02],
         [-7.2895e-02, -5.1334e-03,  8.7359e-02,  ..., -3.0714e-02,
           2.4972e-02, -1.9598e-02],
         [-6.9622e-02, -2.1240e-02,  8.4317e-02,  ..., -2.4805e-02,
           1.8852e-02, -1.6728e-02],
         ...,
         [-5.8537e-02,  3